# Setting things up

## Installation

In [1]:
# %pip install AutoCarver[jupyter]

In [2]:
import sys
import os

print(os.listdir('../../../../../AutoCarver'))

sys.path.append('../../../../../AutoCarver')
sys.path.append('../../../../../AutoCarver/AutoCarver')
sys.path.append('../../../../../AutoCarver/AutoCarver/discretizers')
sys.path.append('../../../../../AutoCarver/AutoCarver/discretizers/utils')
import AutoCarver

['.coverage', '.git', '.github', '.gitignore', '.ipynb_checkpoints', '.pytest_cache', '.readthedocs.yaml', 'AutoCarver', 'AutoCarver.egg-info', 'dist', 'docs', 'LICENSE', 'pyproject.toml', 'README.md', 'requirements.txt', 'setup.cfg', 'setup.py', 'tests', 'test_package.ipynb']


## Titanic Data

In this example notebook, we will use the Titanic dataset.

The Titanic dataset is a well-known and frequently used dataset in the field of machine learning and data science. It provides information about the passengers on board the Titanic, the famous ship that sank on its maiden voyage in 1912. The dataset is often used for predictive modeling, classification, and regression tasks.

The dataset includes various features such as passengers' names, ages, genders, ticket classes, cabin information, and whether they survived or not. The primary goal when working with the Titanic dataset is often to build predictive models that can infer whether a passenger survived or perished based on their individual characteristics (binary classification).

In [3]:
import pandas as pd

# URL to the Titanic dataset on Kaggle
titanic_url = "https://web.stanford.edu/class/archive/cs/cs109/cs109.1166/stuff/titanic.csv"

# Use pandas to read the CSV file directly from the URL
titanic_data = pd.read_csv(titanic_url)

# Display the first few rows of the dataset
titanic_data.head()

Survived  Pclass                                               Name  \
0         0       3                             Mr. Owen Harris Braund   
1         1       1  Mrs. John Bradley (Florence Briggs Thayer) Cum...   
2         1       3                              Miss. Laina Heikkinen   
3         1       1        Mrs. Jacques Heath (Lily May Peel) Futrelle   
4         0       3                            Mr. William Henry Allen   

      Sex   Age  Siblings/Spouses Aboard  Parents/Children Aboard     Fare  
0    male  22.0                        1                        0   7.2500  
1  female  38.0                        1                        0  71.2833  
2  female  26.0                        0                        0   7.9250  
3  female  35.0                        1                        0  53.1000  
4    male  35.0                        0                        0   8.0500

## Target type and Carver selection

In [4]:
target = "Survived"

titanic_data[target].value_counts(dropna=False)

Survived
0    545
1    342
Name: count, dtype: int64

The target ``"Survived"`` is a binary target of type ``int64`` used in a classification task. Hence we will use ``AutoCarver.BinaryCarver`` and ``AutoCarver.selectors.ClassificationSelector`` in following code blocks.

## Data Sampling

In [5]:
from sklearn.model_selection import train_test_split

# stratified sampling by target
train_set, dev_set = train_test_split(titanic_data, test_size=0.33, random_state=42, stratify=titanic_data[target])

c:\Users\defra\.conda\envs\py39\lib\site-packages\sklearn\utils\validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
c:\Users\defra\.conda\envs\py39\lib\site-packages\sklearn\utils\validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


In [6]:
# checking target rate per dataset
train_set[target].mean(), dev_set[target].mean()

(0.38552188552188554, 0.3856655290102389)

# Picking up columns to Carve

In [7]:
train_set.head()

Survived  Pclass                          Name     Sex   Age  \
617         0       3            Mr. Antoni Yasbeck    male  27.0   
489         0       1     Mr. Harry Markland Molson    male  55.0   
871         1       3      Miss. Adele Kiamie Najib  female  15.0   
654         0       3  Mrs. John (Catherine) Bourke  female  32.0   
653         0       3          Mr. Alexander Radeff    male  27.0   

     Siblings/Spouses Aboard  Parents/Children Aboard     Fare  
617                        1                        0  14.4542  
489                        0                        0  30.5000  
871                        0                        0   7.2250  
654                        1                        1  15.5000  
653                        0                        0   7.8958

In [8]:
# column data types
train_set.dtypes

Survived                     int64
Pclass                       int64
Name                        object
Sex                         object
Age                        float64
Siblings/Spouses Aboard      int64
Parents/Children Aboard      int64
Fare                       float64
dtype: object

In [9]:
# values taken by Parents/Children Aboard
train_set["Parents/Children Aboard"].value_counts()

Parents/Children Aboard
0    438
1     87
2     60
3      3
5      3
4      2
6      1
Name: count, dtype: int64

In [10]:
# values taken by Pclass
train_set["Pclass"].value_counts()

Pclass
3    326
1    142
2    126
Name: count, dtype: int64

The feature ``"Pclass"`` is of type ``"int64"``, but it can be considered a qualitative ordinal feature rather than a quantitative discrete feature (ranking of named passenger classes). Thus we will add it to the list of ``ordinal_features`` and set the ordering of its values in ``values_orders`` (string values). 

``"Sex"`` is the only quantitative categorical feature, it's added to the list of ``qualitative_features``.

``"Age"`` and ``"Fare"`` are quantitative continuous features, whilst ``"Siblings/Spouses Aboard"``, ``"Parents/Children Aboard"`` can be considered as quantitative discrete features. Those four features will be added to the list of ``quantitative_features``.

In [11]:
# lists of features per data type
quantitative_features = ["Age", "Fare", "Siblings/Spouses Aboard", "Parents/Children Aboard"]
qualitative_features = ["Sex"]
ordinal_features = ["Pclass"]

# user-specified ordering for ordinal features
values_orders = {
    "Pclass": ["1", "2", "3"]
}

# Using AutoCarver

## AutoCarver settings

### Representativness of modalities

The attribute ``min_freq`` allows one to choose the minimum frequency per basic modalities. It is used by **Discretizers**:

- For quantitative features, it defines the number of quantiles to initialy discretize the features with.

- For qualitative features, it defines the threshold under which a modality is grouped to either a default value or its closest modality.

In [12]:
min_freq = 0.02

**Tip:** should be set between ``0.01`` (slower, preciser, less robust) and ``0.2`` (faster, more robust)

### Desired number of modalities

The attribute ``max_n_mod`` allows one to choose the maximum number of modalities per carved feature. It is used by **Carvers** has the upper limit of number of modalities per consecutive combination of modalities.

In [13]:
max_n_mod = 5

**Tip:** should be set between ``3`` (faster, more robust) and ``7`` (slower, preciser, less robust)

### Association metric

The attribute ``sort_by`` allows one to choose the association metric used to sort combinations. Combinations of grouped modalities are ranked according to the specified modalities and the best ranked viable combination is returned by **Carvers**.

In [14]:
# For BinaryCarver, to be choosen amongst ["tschuprowt", "cramerv"]
sort_by = "tschuprowt"  # "cramerv"

**Tip:** use ``"tschuprowt"`` for more robust, or less output modalities, use ``"cramerv"`` for more output modalities.

### Grouping NaNs

The attribute ``dropna`` allows one to choose whether or not ``numpy.nan`` should be grouped with another modality. If set to ``True``, **Carvers** will first find the most suitable combination of non-NaN values, and then test out all possible combinations with ``numpy.nan``.

In [15]:
dropna = False  # anyway, there are no numpy.nan in this dataset

### Optional attributes

#### Minimal frequency per carved modality

The attribute ``min_freq_mod`` allows one to choose the minimum frequency per output modality. It is used by **Carvers** in viability tests to put aside combinations that are not frequent enough in train or dev sets. By default, it is set to ``min_freq/2``.

In [16]:
min_freq_mod = None  # for 0.05,  at least 5 % of observations per output modality in train and dev sets 

#### Type of output carved features

The attribute ``output_dtype`` allows one to choose the output type:

* Use ``"float"`` for integer output (default)
* Use ``"str"`` for strin output

In [17]:
output_dtype = "float"  # "str"


## Fitting AutoCarver

* First, all qualitative features are discretized:
    1. Using ``StringDiscretizer`` to convert them to ``str`` if not already the case
    2. For qualitative ordinal features: using ``OrdinalDiscretizer`` for under-represented values (less frequent than ``min_freq=0.02``) to be grouped with its closest modality
    3. For qualitative categorical features: using ``CategoricalDiscretizer`` for under-represented values (less frequent than ``min_freq=0.02``) to be grouped with a default value (``str_default="__OTHER__"``)

* Second, all quantitative features are discretized:
    1. Using ``ContinuousDiscretizer`` for quantile discretization that keeps track of over-represented values (more frequent than ``min_freq=0.1``)
    2. Using ``OrdinalDiscretizer`` for any remaining under-represented values (less frequent than ``min_freq/2=0.05``) to be grouped with its closest modality

* Third, all features are carved following this recipe, for all classes of ``train_set[target]`` (except one):
    1. The raw distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the discretization step
    2. Grouping modalities: all consecutive combinations of modalities are applied to ``train_set``
    3. Computing associations: the association metric (``sort_by="cramerv"``) is computed with the provided target ``train_set[target]``
    4. Combinations are sorted in descending order by association value
    5. Testing robustness: finds the first combination that checks the following:
        - Representativness of modalities on ``train_set`` and ``dev_set`` (all should be more frequent than ``min_freq_mod``)
        - Distinct target rates per consecutive modalities on ``train_set`` and ``dev_set`` 
        - No inversion of target rates between ``train_set`` and ``dev_set`` (same ordering of modalities by target rate)
    6. (Optional) If requested via ``dropna=True``, and if any, all combinations of modalities with ``numpy.nan`` are applied to ``train_set`` and steps 3. and 4. are run
    7. The carved distribution is printed out on provided ``train_set`` and ``dev_set``. It's the output of the carving step

In [18]:
from AutoCarver import BinaryCarver

# intiating AutoCarver
auto_carver = BinaryCarver(
    quantitative_features=quantitative_features,
    qualitative_features=qualitative_features,
    ordinal_features=ordinal_features,
    values_orders=values_orders,
    min_freq=min_freq,
    min_freq_mod=min_freq_mod,
    max_n_mod=max_n_mod,
    dropna=dropna,
    sort_by=sort_by,
    output_dtype=output_dtype,
    verbose=True,  # showing statistics
    copy=True,  # whether or not to return a copy of the input dataset
)

# fitting on training sample, a dev sample can be specified to evaluate carving robustness
train_set_processed = auto_carver.fit_transform(train_set, train_set[target], X_dev=dev_set, y_dev=dev_set[target])

------
[Discretizer] Fit Qualitative Features
---
 - [StringDiscretizer] Fit ['Pclass']
 - [OrdinalDiscretizer] Fit ['Pclass']
 - [CategoricalDiscretizer] Fit ['Sex']
------

------
[Discretizer] Fit Quantitative Features
---
 - [ContinuousDiscretizer] Fit ['Age', 'Siblings/Spouses Aboard', 'Fare', 'Parents/Children Aboard']
 - [OrdinalDiscretizer] Fit ['Age', 'Siblings/Spouses Aboard', 'Fare', 'Parents/Children Aboard']
------



c:\Users\defra\Desktop\git\PROJECTS\AutoCarver\docs\source\examples\BinaryClassification\../../../../../AutoCarver\AutoCarver\discretizers\discretizers.py:325: UserWarning:  - [QualitativeDiscretizer] Non-string features: ['Pclass']. Trying to convert them using type_discretizers.StringDiscretizer, otherwise convert them manually. Unexpected data types: [<class 'int'>].
  warn(



------
[AutoCarver] Fit Age (1/6)
---

 - [AutoCarver] Raw distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 2.000e+00 
 0.7500 
 0.0269 
 
 
 2.000e+00 < x <= 4.000e+00 
 0.7143 
 0.0236 
 
 
 4.000e+00 < x <= 7.000e+00 
 0.5000 
 0.0168 
 
 
 7.000e+00 < x <= 1.100e+01 
 0.1667 
 0.0202 
 
 
 1.100e+01 < x <= 1.600e+01 
 0.4400 
 0.0421 
 
 
 1.600e+01 < x <= 1.700e+01 
 0.3000 
 0.0168 
 
 
 1.700e+01 < x <= 1.800e+01 
 0.3333 
 0.0354 
 
 
 1.800e+01 < x <= 1.900e+01 
 0.3913 
 0.0387 
 
 
 1.900e+01 < x <= 2.050e+01 
 0.1111 
 0.0303 
 
 
 2.050e+01 < x <= 2.100e+01 
 0.1905 
 0.0354 
 
 
 2.100e+01 < x <= 2.200e+01 
 0.4242 
 0.0556 
 
 
 2.200e+01 < x <= 2.350e+01 
 0.4000 
 0.0168 
 
 
 2.350e+01 < x <= 2.400e+01 
 0.5417 
 0.0404 
 
 
 2.400e+01 < x <= 2.500e+01 
 0.1333 
 0.0253 
 
 
 2.500e+01 < x <= 2.600e+01 
 0.4000 
 0.0168 
 
 
 2.600e+01 < x <= 2.700e+01 
 0.5000 
 0.0337 
 
 
 2.700e+01 < x <= 2.850e+01 
 0.2500 
 0.0337 
 
 
 2.850e+01 < x <= 2.900e+01 
 0.4444 
 0.0303 
 
 
 2.900e+01 < x <= 3.000e+01 
 0.2917 
 0.0404 
 
 
 3.000e+01 < x <= 3.100e+01 
 0.3846 
 0.0219 
 
 
 3.100e+01 < x <= 3.200e+01 
 0.5000 
 0.0269 
 
 
 3.200e+01 < x <= 3.300e+01 
 0.3846 
 0.0219 
 
 
 3.300e+01 < x <= 3.400e+01 
 0.3077 
 0.0219 
 
 
 3.400e+01 < x <= 3.500e+01 
 0.5000 
 0.0202 
 
 
 3.500e+01 < x <= 3.600e+01 
 0.4375 
 0.0269 
 
 
 3.600e+01 < x <= 3.800e+01 
 0.4118 
 0.0286 
 
 
 3.800e+01 < x <= 3.900e+01 
 0.3333 
 0.0202 
 
 
 3.900e+01 < x <= 4.000e+01 
 0.4615 
 0.0219 
 
 
 4.000e+01 < x <= 4.100e+01 
 0.3333 
 0.0101 
 
 
 4.100e+01 < x <= 4.200e+01 
 0.4615 
 0.0219 
 
 
 4.200e+01 < x <= 4.400e+01 
 0.3333 
 0.0202 
 
 
 4.400e+01 < x <= 4.550e+01 
 0.4167 
 0.0202 
 
 
 4.550e+01 < x <= 4.900e+01 
 0.5200 
 0.0421 
 
 
 4.900e+01 < x <= 5.100e+01 
 0.2727 
 0.0185 
 
 
 5.100e+01 < x <= 5.500e+01 
 0.3846 
 0.0219 
 
 
 5.500e+01 < x <= 5.800e+01 
 0.3636 
 0.0185 
 
 
 5.800e+01 < x <= 6.200e+01 
 0.0909 
 0.0185 
 
 
 6.200e+01 < x 
 0.1818 
 0.0185 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.4444 
 0.0307 
 
 
 0.7500 
 0.0137 
 
 
 0.7500 
 0.0137 
 
 
 0.3750 
 0.0273 
 
 
 0.5000 
 0.0410 
 
 
 0.5000 
 0.0205 
 
 
 0.4000 
 0.0512 
 
 
 0.2000 
 0.0341 
 
 
 0.3333 
 0.0205 
 
 
 0.1538 
 0.0444 
 
 
 0.1667 
 0.0205 
 
 
 0.1875 
 0.0546 
 
 
 0.5000 
 0.0341 
 
 
 0.5000 
 0.0341 
 
 
 0.2727 
 0.0375 
 
 
 0.5000 
 0.0205 
 
 
 0.2632 
 0.0648 
 
 
 0.4286 
 0.0239 
 
 
 0.3333 
 0.0307 
 
 
 0.6250 
 0.0273 
 
 
 0.4000 
 0.0171 
 
 
 0.6667 
 0.0205 
 
 
 0.7500 
 0.0137 
 
 
 0.6000 
 0.0341 
 
 
 0.5714 
 0.0239 
 
 
 0.2500 
 0.0273 
 
 
 0.1667 
 0.0205 
 
 
 0.2000 
 0.0171 
 
 
 0.2000 
 0.0171 
 
 
 0.5000 
 0.0137 
 
 
 0.0000 
 0.0137 
 
 
 0.2500 
 0.0137 
 
 
 0.3636 
 0.0375 
 
 
 0.6667 
 0.0205 
 
 
 0.5000 
 0.0205 
 
 
 0.5000 
 0.0068 
 
 
 0.7500 
 0.0137 
 
 
 0.1429 
 0.0239

Testing robustness    :   0%|          | 1/74518 [00:19<404:51:40, 19.56s/it]


...

X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 0.000e+00 
 0.3447 
 0.7374 
 
 
 0.000e+00 < x <= 1.000e+00 
 0.5057 
 0.1465 
 
 
 1.000e+00 < x <= 2.000e+00 
 0.5167 
 0.1010 
 
 
 2.000e+00 < x 
 0.3333 
 0.0152 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.3475 
 0.8055 
 
 
 0.6774 
 0.1058 
 
 
 0.4500 
 0.0683 
 
 
 0.1667 
 0.0205

Testing robustness    :   0%|          | 0/7 [00:00<?, ?it/s]


 - [AutoCarver] Carved distribution


X distribution 
 
 
   
 target_rate 
 frequency 
 
 
 
 
 x <= 0.000e+00 
 0.3447 
 0.7374 
 
 
 0.000e+00 < x 
 0.5000 
 0.2626 
 
 
 
 
 
 X_dev distribution 
 
 
 target_rate 
 frequency 
 
 
 
 
 0.3475 
 0.8055 
 
 
 0.5439 
 0.1945

------



## AutoCarver analysis

### Carving Summary

In [19]:
auto_carver.summary()

label                       content
feature                 dtype                                     
Age                     float      0              [x <= 4.000e+00]
                        float      1  [4.000e+00 < x <= 5.800e+01]
                        float      2               [5.800e+01 < x]
Parents/Children Aboard float      0              [x <= 0.000e+00]
                        float      1               [0.000e+00 < x]
Siblings/Spouses Aboard float      0              [x <= 0.000e+00]
                        float      1  [0.000e+00 < x <= 2.000e+00]
                        float      2               [2.000e+00 < x]
Pclass                  str        0                        [1, 2]
                        str        1                           [3]
Sex                     str        0                        [male]
                        str        1                      [female]

* As requested with ``output_dtype="float"``, output labels are integers of ranks of modalities

* For quantitative feature ``Age``, the selected combination of modalities groups ages as follows:
    * modality ``0``: lower or equal to 7 years old (``content==["x <= 7.000e+00"]``)
    * modality ``1``: ages higher than 7 years old (``content==["7.000e+00 < x "]``)

* For qualitative categorical feature ``Sex``, the selected combination of modalities has left modalities ``content=["male"]`` in modality ``0`` and ``content=["female"]`` in modality ``1`` (no combination possible)

* For qualitative ordinal feature ``Pclass``, the selected combination of modalities groups classes 1 and 2 in modality ``0`` (``content==[1, 2]``) and class 3 in modality ``1`` (``content==[3]``). The user-provided ordering of modalities has been preserved.

### Detailed overview of tested combinations

In [20]:
auto_carver.history(feature="Pclass")

combination  tschuprowt viability  \
0  [[1, 1], [2, 2], [3, 3]]    0.269965      None   
1    [[1, 1, 2, 2], [3, 3]]    0.300144      True   
2  [[1, 1], [2, 2], [3, 3]]    0.269965      None   
3    [[1, 1], [2, 2, 3, 3]]    0.265643      None   

                          viability_message  grouping_nan  
0                      [Raw X distribution]         False  
1  [Combination robust between X and X_dev]         False  
2                             [Not checked]         False  
3                             [Not checked]         False

* The most associated combination (the first tested out, where ``viability_message!=["Raw X distribution"]``) groups ``Pclass==1`` with ``Pclass==2`` and leaves ``Pclass==3`` as its own modality

* Tschuprow's T for this combination is ``0.300144`` (greater than the raw distribution, with ``0.269965``)

* This combination has been tested as viable: ``viability_message==["Combination robust between X and X_dev"]``

* Following combinations (less associated with the target) where not tested: ``viability_message==["Not checked"]``

* For all combinations ``grouping_nan==False`` means that it is not a combination in which NaNs are being groupedwith other modalities (as requested with ``dropna=False``)

## Saving and Loading AutoCarver

### Saving

All **Carvers** can safely be stored as a .json file.

In [21]:
import json

# storing as json file
with open('binay_carver.json', 'w') as my_carver_json:
    json.dump(auto_carver.to_json(), my_carver_json)

### Loading

**Carvers** can safely be loaded from a .json file.

In [22]:
import json

from AutoCarver import load_carver

# loading json file
with open('binay_carver.json', 'r') as my_carver_json:
    auto_carver = load_carver(json.load(my_carver_json))

## Applying AutoCarver

In [23]:
dev_set_processed = auto_carver.transform(dev_set)

In [24]:
dev_set_processed[auto_carver.features].apply(lambda u: u.value_counts(dropna=False, normalize=True))

Age    Pclass       Sex  Siblings/Spouses Aboard  \
0.0  0.044369  0.450512  0.665529                 0.682594   
1.0  0.918089  0.549488  0.334471                 0.269625   
2.0  0.037543       NaN       NaN                 0.047782   

     Parents/Children Aboard  
0.0                 0.805461  
1.0                 0.194539  
2.0                      NaN

# Feature Selection
## Selectors settings

### Features to select from

Here all features have been carved using ``BinaryCarver``, hence all features are qualitative.

In [25]:
features = qualitative_features + quantitative_features + ordinal_features


### Number of features to select

The attribute ``n_best`` allows one to choose the number of features to be selected per data type (quantitative and qualitative).

In [26]:
n_best = 6  # here the number of features is low, ClassificationSelector will only be used to compute useful statistics

## Using Selectors

In [27]:
from AutoCarver.selectors import ClassificationSelector

# select the most target associated qualitative features
feature_selector = ClassificationSelector(
    qualitative_features=features,
    n_best=n_best,
    verbose=True,  # displays statistics
)
best_features = feature_selector.select(train_set_processed, train_set_processed[target])

------
[Selector] Selecting from qualitative features: ['Age', 'Siblings/Spouses Aboard', 'Sex', 'Pclass', 'Fare', 'Parents/Children Aboard']
---

 - [Selector] Association between X and y


dtype 
 pct_nan 
 pct_mode 
 mode 
 chi2_statistic 
 tschuprowt_measure 
 
 
 
 
 Sex 
 int64 
 0.000000 
 0.636364 
 0 
 169.204709 
 0.533719 
 
 
 Pclass 
 int64 
 0.000000 
 0.548822 
 1 
 53.511433 
 0.300144 
 
 
 Fare 
 float64 
 0.000000 
 0.052189 
 8.050000 
 317.052940 
 0.193792 
 
 
 Siblings/Spouses Aboard 
 int64 
 0.000000 
 0.680135 
 0 
 22.226931 
 0.162663 
 
 
 Age 
 float64 
 0.000000 
 0.912458 
 1.000000 
 21.276115 
 0.159146 
 
 
 Parents/Children Aboard 
 int64 
 0.000000 
 0.737374 
 0 
 11.057602 
 0.136439


 - [Selector] Association between X and y, filtered for inter-feature assocation


dtype 
 pct_nan 
 pct_mode 
 mode 
 chi2_statistic 
 tschuprowt_measure 
 
 
 
 
 Sex 
 int64 
 0.000000 
 0.636364 
 0 
 169.204709 
 0.533719 
 
 
 Pclass 
 int64 
 0.000000 
 0.548822 
 1 
 53.511433 
 0.300144 
 
 
 Fare 
 float64 
 0.000000 
 0.052189 
 8.050000 
 317.052940 
 0.193792 
 
 
 Siblings/Spouses Aboard 
 int64 
 0.000000 
 0.680135 
 0 
 22.226931 
 0.162663 
 
 
 Age 
 float64 
 0.000000 
 0.912458 
 1.000000 
 21.276115 
 0.159146 
 
 
 Parents/Children Aboard 
 int64 
 0.000000 
 0.737374 
 0 
 11.057602 
 0.136439


 - [Selector] Selected qualitative features: ['Sex', 'Pclass', 'Fare', 'Siblings/Spouses Aboard', 'Age', 'Parents/Children Aboard']
------



* Feature ``Sex`` is the most associated with the target ``Survived``. Tschuprow's T value is ``tschuprowt_measure=0.533719``

* This feature has 0 % of NaNs (``pct_nan=0.0``) and its mode, ``0``, represents 64 % of observed data (``pct_nan=0.636364``)

* Here, no feature where filtered out for there inter-feature association or over-represented values (no thresholds were set)